In [1]:
import pandas as pd
import os

# List of CSV files to process
base_path = r'C:\Users\willi\CAN_experiments\\'
csv_files = [
    'performance_metrics_correlated_signal.csv',
    'performance_metrics_max_speedometer.csv',
    'performance_metrics_reverse_light_off.csv',
    'performance_metrics_reverse_light_on.csv',
    'performance_metrics_max_engine_coolant.csv'
]

# Initialize a dictionary to store the data
all_data = {}

# Iterate over each file and load the data
for file in csv_files:
    file_path = os.path.join(base_path, file)
    df = pd.read_csv(file_path)
    attack_type = file.replace('performance_metrics_', '').replace('.csv', '')
    
    # Store the DataFrame in the dictionary
    all_data[attack_type] = df


In [2]:
from scipy.stats import mannwhitneyu, ks_2samp

# Initialize dictionaries to store the test results
mannwhitneyu_results = {}
ks_results = {}

# Perform the tests for each attack type
for attack_type, df in all_data.items():
    embeddings_only = df[df['version'] == 'embeddings_only']['roc_auc']
    normalized = df[df['version'] == 'normalized']['roc_auc']
    
    # Mann-Whitney U test
    u_statistic, p_value_u = mannwhitneyu(embeddings_only, normalized, alternative='less')
    mannwhitneyu_results[attack_type] = {'U Statistic': u_statistic, 'P Value': p_value_u}
    
    # Kolmogorov-Smirnov test
    ks_statistic, p_value_ks = ks_2samp(embeddings_only, normalized)
    ks_results[attack_type] = {'KS Statistic': ks_statistic, 'P Value': p_value_ks}

# Print the results
for attack_type in mannwhitneyu_results:
    print(f"\nResults for '{attack_type}' attack type:")
    print(f"Mann-Whitney U Test - U Statistic: {mannwhitneyu_results[attack_type]['U Statistic']}, P Value: {mannwhitneyu_results[attack_type]['P Value']}")
    print(f"Kolmogorov-Smirnov Test - KS Statistic: {ks_results[attack_type]['KS Statistic']}, P Value: {ks_results[attack_type]['P Value']}")



Results for 'correlated_signal' attack type:
Mann-Whitney U Test - U Statistic: 2824.0, P Value: 5.554342616225123e-16
Kolmogorov-Smirnov Test - KS Statistic: 0.46218487394957986, P Value: 7.623183907227998e-12

Results for 'max_speedometer' attack type:
Mann-Whitney U Test - U Statistic: 4079.0, P Value: 7.984675509937762e-09
Kolmogorov-Smirnov Test - KS Statistic: 0.3277310924369748, P Value: 4.685849564486768e-06

Results for 'reverse_light_off' attack type:
Mann-Whitney U Test - U Statistic: 3128.0, P Value: 4.9766695538893794e-14
Kolmogorov-Smirnov Test - KS Statistic: 0.42857142857142855, P Value: 3.45314884189695e-10

Results for 'reverse_light_on' attack type:
Mann-Whitney U Test - U Statistic: 3343.0, P Value: 9.843705352094683e-13
Kolmogorov-Smirnov Test - KS Statistic: 0.42857142857142855, P Value: 3.45314884189695e-10

Results for 'max_engine_coolant' attack type:
Mann-Whitney U Test - U Statistic: 4297.0, P Value: 8.014861606817912e-08
Kolmogorov-Smirnov Test - KS Statist